In [1]:
import QuantLib as ql

In [9]:
#Create a relinkable yield term structure handle and build a curve
yts = ql.RelinkableYieldTermStructureHandle()

instruments = [
    #('depo','3M', 0.025),
    ('depo','6M', 0.045),
    ('swap','1Y', 0.051),
    ('swap','2Y', 0.071),
    ('swap','3Y', 0.088)
]

helpers = ql.RateHelperVector()
index = ql.Euribor6M(yts)

for inst, tenor, rate in instruments:
    if inst == 'depo':
        helpers.append(ql.DepositRateHelper(rate, index))
    if inst == 'swap':
        swapIndex = ql.EuriborSwapIsdaFixA(ql.Period(tenor))
        helpers.append(ql.SwapRateHelper(rate, swapIndex))

curve = ql.PiecewiseLogLinearDiscount(2, ql.TARGET(), helpers, ql.Actual365Fixed())

In [10]:
#Link the built curve to the relinkable yield term structure handle and build a swap pricing engine

yts.linkTo(curve)
engine = ql.DiscountingSwapEngine(yts)

In [11]:
#Build a vanilla swap and provide a pricing engine
tenor = ql.Period('2Y')
fixedRate = 0.05
forwardStart = ql.Period('2D')

swap = ql.MakeVanillaSwap(tenor, index, fixedRate, forwardStart, Nominal=10e6, pricingEngine=engine)

In [12]:
#Get the fair rate and NPV
fairRate = swap.fairRate()
npv = swap.NPV()

print(f'Fair Swap rate: {fairRate:.3%}')
print(f'Swap NPV: {npv:,.3f}')

Fair Swap rate: 7.133%
Swap NPV: 388,352.123


In [15]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

cashflows = pd.DataFrame({
    'date': cf.date(),
    'amount': cf.amount()
} for cf in swap.leg(1))

display(cashflows)

,date,amount
0,"July 22nd, 2024","228,249.00"
1,"January 22nd, 2025","278,044.81"
2,"July 22nd, 2025","450,768.45"
3,"January 22nd, 2026","461,130.17"


In [17]:
cashflows = pd.DataFrame({
    'nominal': cf.nominal(),
    'accrualStartDate': cf.accrualStartDate().ISO(),
    'accrualEndDate': cf.accrualEndDate().ISO(),
    'rate': cf.rate(),
    'amount': cf.amount()
} for cf in map(ql.as_coupon, swap.leg(1)))

display(cashflows)

,nominal,accrualStartDate,accrualEndDate,rate,amount
0,"10,000,000.00",2024-01-22,2024-07-22,0.05,"228,249.00"
1,"10,000,000.00",2024-07-22,2025-01-22,0.05,"278,044.81"
2,"10,000,000.00",2025-01-22,2025-07-22,0.09,"450,768.45"
3,"10,000,000.00",2025-07-22,2026-01-22,0.09,"461,130.17"
